## Integrated FM Control

In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem import utils
from dataclasses import dataclass, field

microscope, settings = utils.setup_session()

In [ ]:
# Controls
# Light Source 
    # Power
    # Exposure Time
# Filter Wheel
    # Excitation Wavelength
    # Emission Wavelength
# Objective Lens
    # Magnification
    # Position
    # Move Relative
    # Move Absolute
# Camera
    # Binning
    # Gain
    # Offset

# Tasks
# Acquire Image
# Acquire Z-Stack
# Acquire Overview
# Move to Position
# Save OME Metadata
# Load OME Metadata

# TODO:
# - tests
# 


In [ ]:
from typing import Optional
from abc import ABC, abstractmethod
import numpy as np
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
from pprint import pprint

from fibsem.fm.microscope import FluorescenceMicroscope, Camera, LightSource, ObjectiveLens, FilterSet
from fibsem.fm.structures import ChannelSettings, ZParameters
from fibsem.fm.acquisition import acquire_channels, acquire_z_stack, run_auto_focus

# create microscope 
fm = FluorescenceMicroscope()
objective = ObjectiveLens(parent=fm)
filter_set = FilterSet(parent=fm)
camera = Camera(parent=fm)
light_source = LightSource(parent=fm)
fm.objective = objective
fm.filter_set = filter_set
fm.camera = camera
fm.light_source = light_source
# OME-standard -> Static instrument configuration vs dynamic image acquisition settings

zparams = ZParameters(zmin=-10e-6, zmax=10e-6, zstep=1e-6)

ch1 = ChannelSettings(
    name="Channel-01", 
    excitation_wavelength=488.0, 
    emission_wavelength=520.0, 
    power=100.0, 
    exposure_time=200.0, 
    binning=1,
)
ch2 = ChannelSettings(
    name="Channel-02", 
    excitation_wavelength=561.0, 
    emission_wavelength=None, 
    power=80.0, 
    exposure_time=150.0,
    binning=2,
)



In [ ]:
# acquire a single image
image = fm.acquire_image(ch1)
print(image)
print(f"Acquired image with shape: {image.data.shape} and dtype: {image.data.dtype}")
pprint(image.metadata)
plt.imshow(image.data, cmap='gray')
plt.title(f"Acquired Image - {ch1.name}")
plt.colorbar()
plt.show()



In [ ]:
# Acquire images for multiple channels
images = acquire_channels(fm, [ch1, ch2])
for idx, image in enumerate(images):
    print(f"Image {idx+1} Metadata:")
    md = image.metadata
    pprint(md)
    print(f"Image {idx+1} Shape: {image.data.shape}, Dtype: {image.data.dtype}")
    # Optionally display the image
    plt.imshow(image.data, cmap='gray')
    plt.title(f"Acquired Image - {md['objective']['magnification']}x")
    plt.colorbar()
    plt.show()



In [ ]:
# Acquire Z-Stack

fm.objective.move_absolute(0.0)  # Move objective to the starting position
image = acquire_z_stack(fm, ch1, zparams)
print(image[0].data.shape)
md = image[0].metadata

img: np.ndarray
for idx, img in enumerate(image[0].data):
    print(f"Z-Stack Image {idx+1} Metadata:")
    # pprint(md[idx])
    print(f"Z-Stack Image {idx+1} Shape: {img.shape}, Dtype: {img.dtype}")
    # Optionally display the image
    plt.imshow(img, cmap='gray')
    plt.title(f"Z-Stack Image - Z Position: {md['objective']['position']:.2e} microns")
    plt.colorbar()
    plt.show()


In [ ]:
# Run autofocus
fm.objective.move_absolute(0.0)  # Reset objective position to 0
best_focus = run_auto_focus(fm, ch1)
print(f"Best focus position: {best_focus:.2e} microns")

## Odemis FM Control

In [ ]:
%load_ext autoreload
%autoreload 2

import fibsem.fm.odemis as fm_odemis
from odemis import model

focuser = fm_odemis.model.getComponent(role="focus")

print(focuser.position.value)
print(focuser.getMetadata())


In [ ]:
objective = fm_odemis.OdemisObjectiveLens(None)

print(objective.position)
print(objective.magnification)
print(objective.numerical_aperture)

In [ ]:
objective.insert()
print(objective.position)
objective.retract()
print(objective.position)

In [ ]:
odemis_light_source = model.getComponent(role="light")
print(odemis_light_source.power.value)
print("SPECTRA")
print(odemis_light_source.spectra.value) # 5D (99% low, 25% low, centre, 25%high 99% high) x wavelengths

In [ ]:
fm = fm_odemis.OdemisFluorescenceMicroscope(None)

# set filter wheel
fm.filter_set.emission_wavelength = 488e-9
print(fm.filter_set.emission_wavelength)

# reflection mode
fm.filter_set.emission_wavelength = None
print(fm.filter_set.emission_wavelength)

In [ ]:
from fibsem.fm.structures import ChannelSettings
from pprint import pprint


ch = ChannelSettings(name="channel-01", 
                     excitation_wavelength=405e-9, 
                     emission_wavelength=None, 
                     exposure_time=0.5, 
                     power=0.5, # W 
                     binning=4)

fm.set_binning(2)
fm.set_power(0.8)

image = fm.acquire_image(None)

pprint(image.data.shape)
pprint(image.metadata)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from pprint import pprint
plt.imshow(image.data, cmap="gray")
plt.show()


print(image.data.shape)
pprint(image.metadata)
pprint(fm.get_metadata())

In [ ]:
from fibsem.fm.acquisition import acquire_channels, acquire_z_stack, run_auto_focus
from fibsem.fm.structures import ZParameters
zparams = ZParameters(zmin=-10e-6, zmax=10e-6, zstep=1e-6)

fm.objective.insert()
image = acquire_z_stack(fm, ch, zparams)

print(image[0].data.shape)
pprint(image[0].metadata)


In [ ]:
ch1 = ChannelSettings(name="channel-01", 
                     excitation_wavelength=405e-9, 
                     emission_wavelength=None, 
                     exposure_time=0.5, 
                     power=0.5, # W 
                     binning=1)

ch2 = ChannelSettings(name="channel-02",
                     excitation_wavelength=488e-9, 
                     emission_wavelength=500e-9,
                     exposure_time=0.1, 
                     power=0.1, # W 
                     binning=1)

channels = [ch1, ch2]
images = acquire_channels(fm, channels)

for img in images:
    print(img.data.shape)
    pprint(img.metadata)
    plt.imshow(img.data, cmap="gray")
    plt.show()

In [ ]:
# images = acquire_z_stack(fm, ch1, zparams)
# print(images[0].data.shape)

images = acquire_z_stack(fm, channels, zparams)

print(len(images))
print(images[0].data.shape)

In [ ]:
for img in images:
    print(img.data.shape)
    pprint(img.metadata)
    # plt.imshow(img.data, cmap="gray")
    # plt.show()